### Lab2 : Working with Spark SQL

#### We will review :

1. Loading CSV file formats using SparkSession
2. Creating DataFrame without inferring Schema 
3. Creating DataFrame inferring Schema 
4. Doing some preliminary analysis using Spark SQL on this dataset
5. Creating UDFs (User Defined Functions) and using them on the dataset
5. Saving a DataFrame into partitioned parquet files format

#### Small (Lab) Dataset :

* Air flight data - subset of ~ 100 MB (for demonstration purposes)
* Available in the IE cluster @: /data/shared/spark/flight_data/csv_tiny

#### Larger Dataset (Further Labs) :

* Air flight data - subset of ~ 2.5 GB (for cluster operation purposes)
* Available in the IE cluster @: /data/shared/spark/flight_data/csv_small


In [1]:
# First Let's start by :
# 1. Definining SPARK_HOME variable 
# 2. Using findspark to  let us work with Spark installation in the cluster

In [2]:
import os
print(os.environ['SPARK_HOME'])

/usr/hdp/current/spark2-client


In [3]:
import findspark
findspark.init()
import pyspark

In [4]:
# Create a SparkSession and specify configuration
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .master("local[1]") \
    .appName("Spark-SQL-Lab2") \
    .getOrCreate()

In [5]:
dataset_path="/data/shared/spark/flight_data/csv_tiny/"

In [6]:
# Read in all available data files into a data frame
df = spark.read \
    .csv("file://"+dataset_path+"*.csv")   

### Now check the data schema

In [7]:
df.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: string (nullable = true)
 |-- _c4: string (nullable = true)
 |-- _c5: string (nullable = true)
 |-- _c6: string (nullable = true)
 |-- _c7: string (nullable = true)
 |-- _c8: string (nullable = true)
 |-- _c9: string (nullable = true)
 |-- _c10: string (nullable = true)
 |-- _c11: string (nullable = true)
 |-- _c12: string (nullable = true)
 |-- _c13: string (nullable = true)
 |-- _c14: string (nullable = true)
 |-- _c15: string (nullable = true)
 |-- _c16: string (nullable = true)
 |-- _c17: string (nullable = true)
 |-- _c18: string (nullable = true)
 |-- _c19: string (nullable = true)
 |-- _c20: string (nullable = true)
 |-- _c21: string (nullable = true)
 |-- _c22: string (nullable = true)
 |-- _c23: string (nullable = true)
 |-- _c24: string (nullable = true)
 |-- _c25: string (nullable = true)
 |-- _c26: string (nullable = true)
 |-- _c27: string (nullable = tru

* Ok , but the column names are not very telling. 
* How to improve this? , by telling Spark to use the header ( if exists )

In [8]:
df = spark.read \
    .option("header", "true") \
    .csv("file://"+dataset_path+"*.csv")

In [9]:
df.printSchema()

root
 |-- Year: string (nullable = true)
 |-- Quarter: string (nullable = true)
 |-- Month: string (nullable = true)
 |-- DayofMonth: string (nullable = true)
 |-- DayOfWeek: string (nullable = true)
 |-- FlightDate: string (nullable = true)
 |-- UniqueCarrier: string (nullable = true)
 |-- AirlineID: string (nullable = true)
 |-- Carrier: string (nullable = true)
 |-- TailNum: string (nullable = true)
 |-- FlightNum: string (nullable = true)
 |-- OriginAirportID: string (nullable = true)
 |-- OriginAirportSeqID: string (nullable = true)
 |-- OriginCityMarketID: string (nullable = true)
 |-- Origin: string (nullable = true)
 |-- OriginCityName: string (nullable = true)
 |-- OriginState: string (nullable = true)
 |-- OriginStateFips: string (nullable = true)
 |-- OriginStateName: string (nullable = true)
 |-- OriginWac: string (nullable = true)
 |-- DestAirportID: string (nullable = true)
 |-- DestAirportSeqID: string (nullable = true)
 |-- DestCityMarketID: string (nullable = true)
 |-

* Better , but still one caveat though , all values are interpreted as string, while some of them (actually most), are of numeric nature ( e.g ) Year , Month , Flight Number
* How to improve this ?, by either telling Spark what schema to use OR telling it to infer the Schema of the data
* Note : Asking Spark to infer schema may have a performance impact depending on the number of rows required to infer the schema

In [10]:
df = spark.read \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .csv("file://"+dataset_path+"*.csv")

In [11]:
df.printSchema()

root
 |-- Year: integer (nullable = true)
 |-- Quarter: integer (nullable = true)
 |-- Month: integer (nullable = true)
 |-- DayofMonth: integer (nullable = true)
 |-- DayOfWeek: integer (nullable = true)
 |-- FlightDate: timestamp (nullable = true)
 |-- UniqueCarrier: string (nullable = true)
 |-- AirlineID: integer (nullable = true)
 |-- Carrier: string (nullable = true)
 |-- TailNum: string (nullable = true)
 |-- FlightNum: integer (nullable = true)
 |-- OriginAirportID: integer (nullable = true)
 |-- OriginAirportSeqID: integer (nullable = true)
 |-- OriginCityMarketID: integer (nullable = true)
 |-- Origin: string (nullable = true)
 |-- OriginCityName: string (nullable = true)
 |-- OriginState: string (nullable = true)
 |-- OriginStateFips: integer (nullable = true)
 |-- OriginStateName: string (nullable = true)
 |-- OriginWac: integer (nullable = true)
 |-- DestAirportID: integer (nullable = true)
 |-- DestAirportSeqID: integer (nullable = true)
 |-- DestCityMarketID: integer (nu

In [12]:
df.columns

['Year',
 'Quarter',
 'Month',
 'DayofMonth',
 'DayOfWeek',
 'FlightDate',
 'UniqueCarrier',
 'AirlineID',
 'Carrier',
 'TailNum',
 'FlightNum',
 'OriginAirportID',
 'OriginAirportSeqID',
 'OriginCityMarketID',
 'Origin',
 'OriginCityName',
 'OriginState',
 'OriginStateFips',
 'OriginStateName',
 'OriginWac',
 'DestAirportID',
 'DestAirportSeqID',
 'DestCityMarketID',
 'Dest',
 'DestCityName',
 'DestState',
 'DestStateFips',
 'DestStateName',
 'DestWac',
 'CRSDepTime',
 'DepTime',
 'DepDelay',
 'DepDelayMinutes',
 'DepDel15',
 'DepartureDelayGroups',
 'DepTimeBlk',
 'TaxiOut',
 'WheelsOff',
 'WheelsOn',
 'TaxiIn',
 'CRSArrTime',
 'ArrTime',
 'ArrDelay',
 'ArrDelayMinutes',
 'ArrDel15',
 'ArrivalDelayGroups',
 'ArrTimeBlk',
 'Cancelled',
 'CancellationCode',
 'Diverted',
 'CRSElapsedTime',
 'ActualElapsedTime',
 'AirTime',
 'Flights',
 'Distance',
 'DistanceGroup',
 'CarrierDelay',
 'WeatherDelay',
 'NASDelay',
 'SecurityDelay',
 'LateAircraftDelay',
 'FirstDepTime',
 'TotalAddGTime',
 

In [13]:
# Register a table named flights for later SQL queries
df.registerTempTable("flights")

#### Worth Noting

* registerTempTable() creates an in-memory table avaialble within cluster in which it was created. The data is stored using Hive's in-memory columnar format and will only 'live' for the duration of the session.

* saveAsTable() creates a permanent, physical table stored using the Parquet format. This table is accessible to all clusters including external clusters and in between sessions. The table metadata including the location of the file(s) is stored within the Hive metastore.

#### Select the following columns from the full dataset

    Year
    Month
    DayOfMonth
    DayOfWeek
    FlightNum
    Origin
    Carrier
    Dest ( destination )
    DepTime ( departure time )
    DepDelayMinutes ( departure delay )
    ArrTime ( arrival time )
    ArrDelay ( arrival delay )
    Cancelled
    CancellationCode
    AirTime
    Distance


In [14]:
df_subset=spark.sql(
    "select " 
    +"year,month,dayofmonth,dayofweek,"
    +"flightnum,origin,carrier,dest,deptime,depdelay,"
    +"arrtime,arrdelay,cancelled,cancellationcode,"
    +"airtime,distance "
    +"FROM flights"
    )
# OR 
# selection=["year,month,dayofmonth,dayofweek,"
#       "flightnum,origin,dest,deptime,depdelay,"
#       "arrtime,arrdelay,cancelled,cancellationcode,"
#       "airtime,distance "]
# info.select(selection)

In [15]:
# Cache this DataFrame 
df_subset.cache()
# Cache the flights table
spark.sql("cache table flights")

DataFrame[]

In [16]:
# Show the first 5 rows of the subset data to get a feeling of what to expect
df_subset.head(5)

[Row(year=2014, month=9, dayofmonth=1, dayofweek=1, flightnum=1, origin='JFK', carrier='AA', dest='LAX', deptime=851, depdelay=-9.0, arrtime=1144, arrdelay=-26.0, cancelled=0.0, cancellationcode=None, airtime=325.0, distance=2475.0),
 Row(year=2014, month=9, dayofmonth=2, dayofweek=2, flightnum=1, origin='JFK', carrier='AA', dest='LAX', deptime=902, depdelay=2.0, arrtime=1210, arrdelay=0.0, cancelled=0.0, cancellationcode=None, airtime=312.0, distance=2475.0),
 Row(year=2014, month=9, dayofmonth=3, dayofweek=3, flightnum=1, origin='JFK', carrier='AA', dest='LAX', deptime=849, depdelay=-11.0, arrtime=1215, arrdelay=5.0, cancelled=0.0, cancellationcode=None, airtime=330.0, distance=2475.0),
 Row(year=2014, month=9, dayofmonth=4, dayofweek=4, flightnum=1, origin='JFK', carrier='AA', dest='LAX', deptime=852, depdelay=-8.0, arrtime=1133, arrdelay=-37.0, cancelled=0.0, cancellationcode=None, airtime=316.0, distance=2475.0),
 Row(year=2014, month=9, dayofmonth=5, dayofweek=5, flightnum=1, ori

In [17]:
df_subset.take(3)

[Row(year=2014, month=9, dayofmonth=1, dayofweek=1, flightnum=1, origin='JFK', carrier='AA', dest='LAX', deptime=851, depdelay=-9.0, arrtime=1144, arrdelay=-26.0, cancelled=0.0, cancellationcode=None, airtime=325.0, distance=2475.0),
 Row(year=2014, month=9, dayofmonth=2, dayofweek=2, flightnum=1, origin='JFK', carrier='AA', dest='LAX', deptime=902, depdelay=2.0, arrtime=1210, arrdelay=0.0, cancelled=0.0, cancellationcode=None, airtime=312.0, distance=2475.0),
 Row(year=2014, month=9, dayofmonth=3, dayofweek=3, flightnum=1, origin='JFK', carrier='AA', dest='LAX', deptime=849, depdelay=-11.0, arrtime=1215, arrdelay=5.0, cancelled=0.0, cancellationcode=None, airtime=330.0, distance=2475.0)]

### Do some SQL queries ( use both the DataFrame and direct SQL queries )

1. Find the number of departing flights from a given airport
2. Find the total number of delayed flights on a given airport
3. Find the average delay per airport
4. Find the top 5 airports with the highest average delays
5. Find the worst airport in terms of total nb cancelled flights (cancelled=1.0) 

In [18]:
# how many records do we have in total?
total=df_subset.count()
print('Total nb.of flights: %d' % total)
# OR in SQL
spark.sql("select COUNT(*) from flights").show()

Total nb.of flights: 469489
+--------+
|count(1)|
+--------+
|  469489|
+--------+



In [19]:

# 1.2. how many flights and delayed
def statsByAirport(airport_id,df):
    from_id=df.filter(df['origin']==airport_id)
    delayed=from_id.filter(df['depdelay']>=15.0)
    ndep=from_id.count()
    ndel=delayed.count()
    return (ndep,ndel)
    
airport='JFK'

n,m=statsByAirport(airport,df_subset)

print('Departing from %s : %d ' %(airport,n))
print('Delayed   from %s : %d ' %(airport,m))
print('Delayed Percentage : %f %%' %((m/n)*100))

Departing from JFK : 8275 
Delayed   from JFK : 1033 
Delayed Percentage : 12.483384 %


In [20]:
# 3. Average delay per flight on an airport
def averageDelay(airport_id,df):
    from_id=df.filter(df['origin']==airport_id)
    return from_id.select('depdelay').describe() # returns a dataframe with descriptive statistics

airport='JFK'
df=averageDelay(airport,df_subset)

print('Airport : %s ' %(airport))
print('Average delay : %f min' %(float(df.collect()[1]['depdelay'])))

Airport : JFK 
Average delay : 3.856346 min


In [21]:
# 4. Top 5 airports with highest average delay : actually easier here with SQL AVG
query = "SELECT origin,AVG(depdelay) FROM flights GROUP BY origin ORDER BY avg(depdelay) DESC"
df_delays=spark.sql(query)
df_delays.show()

+------+------------------+
|origin|     avg(depdelay)|
+------+------------------+
|   BGR|36.166666666666664|
|   HDN|              34.0|
|   UST|33.241379310344826|
|   OTH|           23.8125|
|   CEC|             23.15|
|   STC|22.962264150943398|
|   CMX|20.847826086956523|
|   MQT|             18.28|
|   ERI| 18.11320754716981|
|   SCE| 17.10909090909091|
|   PPG|16.555555555555557|
|   DBQ|16.486842105263158|
|   CIC|16.453488372093023|
|   FAY|16.271084337349397|
|   JAN| 15.46089850249584|
|   BKG|14.866666666666667|
|   CSG|14.789473684210526|
|   GUM|14.689655172413794|
|   BRO|14.677248677248677|
|   MDT|14.676991150442477|
+------+------------------+
only showing top 20 rows



In [22]:
# 5. The worst airport in terms of cancelled flights
# Create a function that simply sums the total number of flights cancelled on a given airport
# ---> Remember you should 'weight' cancelled against total , in order not to bias the result
#
def cancellations(airport_id):
    # USING DF: 
    #     idf=df_subset.filter(df_subset['origin']==airport_id)
    #     return idf.filter(idf['cancelled']==1.0).count()
    # USING SQL:
    query="select * from flights where origin=='"+airport_id+"'" +" and cancelled==1.0"
    return spark.sql(query).select('cancelled').count()

In [ ]:
# Register the function with Spark SQL as User Defined Function
spark.udf.register("cancellations", lambda x : cancellations(x))
query = "SELECT origin,cancellations(origin) AS score FROM flights GROUP BY origin ORDER BY score DESC"
spark.sql(query).show()

* Save this dataframe in parquet (columnar) format for boost in loading performance
* In order to do we want to 'be clever' and partition the data by specific atributes , in this case
* Year and Month

In [ ]:
# Save the data into my HOME
# IMPORTANT NOTE: we are partinioning (structuring)
# by relevant factors in our data , in this case year and month
# can be used to naturally save this data.
my_home=os.environ['HOME']
out_dir="airline_data"
df_subset.write.partitionBy(
        "Year","Month"
    ).parquet(
        "file://"
        + my_home
        +'/'
        + out_dir,
        mode='overwrite'
    )
print('Completed!')

In [ ]:
# Read CSV data into a dictionary of DataFrame : try to infer schema directly from the data
import itertools
year_list = ['2014']
month_list = ['1','2','3','4','5','6','7','8','9','10','11','12']

dict_df = {}

for (year_str,month_str) in list(itertools.product(year_list,month_list)):
    year_month_str = '%s_%s'%(year_str,month_str)
    print('Reading input data for year:%s month:%s'%(year_str,month_str))
    df = spark.read \
        .option("header", "true") \
        .option("inferSchema", "true") \
        .csv("file://"+dataset_path+"On_Time_On_Time_Performance_%s.csv"%(year_month_str))  
    df.cache()
    dict_df[year_month_str]=df
print('Completed!')